# Estimaciones de Riesgo Extremo

#### Librerías a usar

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats 
import edhec_risk_kit as erk

---

## I. Supuesto de Normalidad en Retornos Financieros

Una asunción común en modelos financieros es que los retornos siguen una distribución normal. Esto implica que los eventos extremos (grandes pérdidas o ganancias) son raros y ocurren con baja probabilidad.

**Sin embargo, en los mercados reales:**
- Las distribuciones de retornos suelen tener **colas gruesas** (más eventos extremos).
- También pueden presentar **asimetría**, es decir, tener más probabilidad de eventos extremos en una dirección (ganancias o pérdidas).

Por tanto, es fundamental utilizar métricas que capturen estos comportamientos no normales para medir el riesgo de manera adecuada.

---

## II. Skewness o Coeficiente de Asimetría

La **skewness** mide la asimetría de una distribución con respecto a su media.

$$
S(X) = \frac{\mathbb{E}[(X - \mu)^3]}{\sigma^3}
$$

Donde:
- $\mu$: media de $X$
- $\sigma$: desviación estándar de $X$

**Interpretación:**
- $ S = 0 $: distribución simétrica (como la normal)
- $ S > 0 $: sesgo a la derecha (cola más pesada a la derecha)
- $ S < 0 $: sesgo a la izquierda (cola más pesada a la izquierda)

La skewness permite identificar si los retornos tienen una mayor probabilidad de producir pérdidas extremas o ganancias extremas.

---

## III. Kurtosis o Curtosis

La **kurtosis** mide el grado de concentración de los datos en los extremos de la distribución (colas).

$$
K(X) = \frac{\mathbb{E}[(X - \mu)^4]}{\sigma^4}
$$

**Interpretación:**
- $Kurtosis = 3$: distribución normal
- $Kurtosis > 3$: colas más pesadas (más eventos extremos) → leptocúrtica
- $Kurtosis < 3$: colas más livianas → platicúrtica

Se suele utilizar la **exceso de kurtosis**: $ K - 3 $, para compararla directamente con la normal.

**Importante:** la kurtosis no mide directamente el "pico" de la distribución, sino la frecuencia de valores extremos.

---

## IV. Prueba de Jarque-Bera (JB)

La prueba de **Jarque-Bera** combina skewness y kurtosis para evaluar si una variable aleatoria sigue una distribución normal.

$$
JB = \frac{n}{6} \left( S^2 + \frac{(K - 3)^2}{4} \right)
$$

Donde:
- $ n $: número de observaciones
- $ S $: skewness
- $ K $: kurtosis

**Hipótesis:**
- $ H_0 $: la variable tiene distribución normal
- $ H_1 $: la variable no tiene distribución normal

Si el estadístico JB es suficientemente grande, se rechaza $ H_0 $.

---

## V. Semivolatilidad (o Semidesviación)

La semivolatilidad mide la variabilidad **solo de los retornos por debajo de un umbral**, típicamente la media o cero. Es útil en finanzas porque los inversionistas suelen preocuparse más por la volatilidad negativa (pérdidas).

$$
\sigma_{semi} = \sqrt{ \frac{1}{N_{semi}} \sum_{R_t < \text{umbral}} (R_t - \mu)^2 }
$$

- Si el umbral es 0 → se mide la volatilidad de las pérdidas netas
- Si el umbral es $ \mu $ → se mide la volatilidad respecto a la media

---

## VI. Valor en Riesgo (VaR)

El **VaR (Value-at-Risk)** mide la pérdida máxima esperada en un horizonte de tiempo dado con un nivel de confianza $ \alpha $.

$$
\text{VaR}_\alpha = - \inf \left\{ x \in \mathbb{R} \,:\, \mathbb{P}(R \leq x) \geq 1 - \alpha \right\}
$$

**Interpretación:**  
Con un nivel de confianza \( \alpha \), el VaR representa la **pérdida mínima** en el peor \( (1 - \alpha)\% \) de los casos.

**Ejemplo:**  
Un VaR al 95% de 4% implica que hay un 5% de probabilidad de perder **más del 4%** en un día determinado.

**Importante:** aunque el retorno es negativo, el VaR se reporta en valor absoluto como pérdida positiva.

---

## VII. Valor en Riesgo Condicional (CVaR o Expected Shortfall)

El **CVaR (Conditional Value-at-Risk)** mide la **pérdida esperada en los peores casos**, es decir, **el promedio de las pérdidas más allá del VaR**.

$$
\text{CVaR}_\alpha = - \mathbb{E}( R \mid R < -\text{VaR}_\alpha )
$$

Es una medida más conservadora que el VaR, ya que no solo considera el umbral de pérdida, sino también su severidad.

---

## VIII. Métodos para estimar VaR y CVaR

### 1. Método Histórico (No paramétrico)

- Se ordenan los retornos observados y se calcula el percentil $ (1 - \alpha) $.
- No requiere ninguna suposición sobre la forma de la distribución.
- CVaR se calcula como el promedio de los retornos que están por debajo del VaR.


### 2. Método Paramétrico (asumiendo normalidad)

- Supone que los retornos siguen una distribución normal con media $ \mu $ y desviación estándar $ \sigma $.
- VaR se calcula como:

$$
\text{VaR}_\alpha = - \left( \mu + z_\alpha \cdot \sigma \right)
$$

Donde:
- $ z_\alpha $: cuantil $ (1 - \alpha) $ de la normal estándar

Este método es sencillo pero poco preciso cuando los retornos presentan colas gruesas o asimetrías.


### 3. Método de Cornish-Fisher (Semi-paramétrico)

- Ajusta el cuantil $ z_\alpha $ para reflejar la skewness y kurtosis de la distribución.
- Utiliza una expansión de Cornish-Fisher para aproximar los cuantiles corregidos:

$$
\tilde{z}_\alpha = z_\alpha + \frac{1}{6}(z_\alpha^2 - 1)S 
+ \frac{1}{24}(z_\alpha^3 - 3 z_\alpha)(K - 3) 
- \frac{1}{36}(2z_\alpha^3 - 5z_\alpha)S^2
$$

$$
\text{VaR}_\alpha = - (\mu + \tilde{z}_\alpha \cdot \sigma)
$$

Este método ofrece estimaciones más realistas cuando los datos no siguen una distribución normal (como t-student).

---